In [1]:
import os
import pandas as pd
import torch
import re
from tqdm import tqdm
from soynlp.normalizer import repeat_normalize
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, BertTokenizer, BertForSequenceClassification, GPT2LMHeadModel, GPT2Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import numpy as np
from datetime import datetime
from konlpy.tag import Okt


In [2]:

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터셋 로드 및 전처리
file_path = './외국음식전문점.csv'
df = pd.read_csv(file_path)
df['content'] = df['content'].fillna('')
text_data = df['content'].tolist()


In [3]:

# 전처리 함수 정의
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^가-힣\s]', '', text)
    text = repeat_normalize(text, num_repeats=2)
    stopwords = ['이', '그', '저', '의', '을', '를', '은', '는', '에', '와', '과', '도', '으로', '그리고', '하지만', '그래서']
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text.strip()

preprocessed_data = [preprocess_text(review) for review in text_data]


In [4]:

# 카테고리 및 강화된 키워드 설정
categories = ['맛', '가격', '서비스', '분위기', '추천', '비추천']
category_keywords = {
    '맛': ['맛', '요리', '음식', '메뉴', '향', '식감', '맛집', '고소', '달콤', '짭짤', '신선', '풍미', '스파이시', '향긋', '촉촉', '바삭', '탱탱', '육즙', '알싸', '상큼', '쫄깃'],
    '가격': ['가격', '비싸', '저렴', '가성비', '비용', '비교', '비율', '가격대', '할인', '구성', '합리적', '저렴하다', '가격 대비', '적당', '경제적', '비용 절감', '가치', '수익', '돈값', '고가', '저가'],
    '서비스': ['서비스', '직원', '응대', '친절', '매너', '대응', '접대', '환대', '서빙', '응대', '고객', '케어', '친절하다', '프로페셔널', '빠르다', '세심하다', '친절도', '고객 관리', '서비스 정신', '상냥', '대접'],
    '분위기': ['분위기', '환경', '인테리어', '공간', '장식', '조명', '음악', '장소', '배경', '아늑하다', '세련', '깔끔', '모던', '클래식', '고급스럽다', '편안하다', '우아', '로맨틱', '분위기 있는', '차분하다', '힐링'],
    '추천': ['추천', '소개', '다음에 또', '재방문', '다시 방문', '친구', '소개하고 싶다', '꼭 오고 싶다', '꼭 다시', '나만 알고 싶다'],
    '비추천': ['비추천', '다신 안', '절대 안', '다시 오지 않다', '다신 안 가다', '비추', '실망', '후회', '최악']
}


In [5]:

# 리뷰 카테고리 분류 함수
def categorize_review(review):
    tfidf_vectorizer = TfidfVectorizer()
    keyword_docs = [" ".join(keywords) for keywords in category_keywords.values()]
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_docs + [review])
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()
    top_categories = [categories[i] for i in cosine_similarities.argsort()[-2:]]
    return top_categories


In [6]:

# 감정 분석 모델 로드
tokenizer_kobert = BertTokenizer.from_pretrained('monologg/kobert')
model_kobert = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=3).to(device)  # 3개의 레이블: 긍정, 부정, 중립

# 감정 분석 수행 및 카테고리별 비율 분석
sentiment_results = []
category_sentiments = {category: Counter() for category in categories}

for review in tqdm(preprocessed_data, desc="Processing Reviews"):
    inputs_kobert = tokenizer_kobert(review, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        output_kobert = model_kobert(**inputs_kobert)
        sentiment = torch.argmax(output_kobert.logits, dim=1).item()
        sentiment_results.append(sentiment)

    categories_found = categorize_review(review)
    for category in categories_found:
        category_sentiments[category][sentiment] += 1


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
C:\Users\G-01\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Processing

In [7]:

# 대표 메뉴 추출 (가게별로)
def extract_top_menus(reviews, top_n=5):
    okt = Okt()
    menu_counter = Counter()
    for review in reviews:
        nouns = okt.nouns(review)  # 리뷰에서 명사만 추출
        menu_counter.update(nouns)  # 명사 빈도를 세어줌
    return menu_counter.most_common(top_n)  # 상위 top_n개의 대표 메뉴 추출

# 리뷰 요약 모델 로드 및 요약 생성 (간결한 요약 설정)
bart_tokenizer = PreTrainedTokenizerFast.from_pretrained('EbanLee/kobart-summary-v3')
bart_model = BartForConditionalGeneration.from_pretrained('EbanLee/kobart-summary-v3').to(device)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [8]:

def summarize_reviews(reviews):
    combined_reviews = " ".join(reviews)
    
    # 1차 요약
    inputs_bart = bart_tokenizer([combined_reviews], return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    summary_ids = bart_model.generate(
        inputs_bart['input_ids'], 
        attention_mask=inputs_bart['attention_mask'], 
        max_length=50,  # 더 짧은 요약
        min_length=20,  
        length_penalty=2.0,  
        num_beams=4, 
        no_repeat_ngram_size=2,  
        repetition_penalty=1.2,  
        early_stopping=True
    )
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary


In [9]:

# 자연스러움 개선을 위한 GPT 모델 로드
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

def refine_summary(summary):
    # GPT-2를 활용해 문장의 자연스러움 개선
    inputs = gpt_tokenizer.encode(summary, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = gpt_model.generate(inputs, max_length=100, num_beams=5, early_stopping=True)
    refined_summary = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return refined_summary


In [11]:
# 가게별 리뷰 종합 요약 및 점수 계산

# 그룹화된 데이터를 가게별로 가져오기
store_grouped = df.groupby('store_name')['content']

store_summaries = []

# 순차적으로 가게별 리뷰를 요약 생성 및 점수 계산
for store_name, reviews in tqdm(store_grouped, desc="Processing Stores"):
    try:
        reviews = reviews.tolist()  # 그룹화된 데이터는 시리즈로 나오므로 리스트로 변환
        num_reviews = len(reviews)
        
        if num_reviews < 50:
            # 리뷰가 50개 미만인 경우에도 데이터가 부족할 수 있다는 설명 추가
            store_summaries.append({
                'Store Name': store_name,
                'Summary': '리뷰가 50개 미만이어서 데이터가 부족할 수 있습니다.',
                'Negative Ratio': sentiment_ratios.get('Negative', 'N/A'),
                'Neutral Ratio': sentiment_ratios.get('Neutral', 'N/A'),
                'Positive Ratio': sentiment_ratios.get('Positive', 'N/A'),
                'Taste Score': category_scores.get('Taste', 'N/A'),
                'Price Score': category_scores.get('Price', 'N/A'),
                'Service Score': category_scores.get('Service', 'N/A'),
                'Atmosphere Score': category_scores.get('Atmosphere', 'N/A'),
                'Recommendation Score': category_scores.get('Recommendation', 'N/A'),
                'Non-recommendation Score': category_scores.get('Non-recommendation', 'N/A'),
                'Overall Score': 'N/A',
                'Top Menus': 'N/A'
            })
            continue

        preprocessed_reviews = [preprocess_text(review) for review in reviews]
        
        # 리뷰 요약
        summary = summarize_reviews(preprocessed_reviews)
        
        # 요약된 문장을 자연스럽게 다듬음
        refined_summary = refine_summary(summary)

        # 감성 비율 계산
        sentiments = [sentiment_results[i] for i, r in enumerate(preprocessed_data) if r in reviews]
        sentiment_counts = Counter(sentiments)
        total_reviews = len(sentiments)
        sentiment_ratios = {}

        # 'Negative', 'Neutral', 'Positive' 레이블에 해당하는 인덱스 값을 설정합니다.
        labels = ['Negative', 'Neutral', 'Positive']

        # 각 레이블에 대해 루프를 돌며 감정 비율을 계산합니다.b
        for i, label in enumerate(labels):
            # 특정 감정의 리뷰 수를 가져옵니다. 없다면 0을 반환합니다.
            count = sentiment_counts.get(i, 0)
            
            # 전체 리뷰 수에 대한 해당 감정의 비율을 계산합니다.
            ratio = count / total_reviews if total_reviews > 0 else 0
            
            # 계산된 비율을 해당 레이블에 맞춰 딕셔너리에 추가합니다.
            sentiment_ratios[label] = ratio

        # 카테고리별 점수 계산
        category_scores = {}
        for category in categories:
            category_count = sum(category_sentiments[category].values())
            category_scores[category] = category_sentiments[category][2] / category_count if category_count > 0 else 0  # 긍정 점수

        # 대표 메뉴 추출
        top_menus = extract_top_menus(preprocessed_reviews)

        # 최종 점수 계산 (예: 감성 비율과 카테고리 점수의 평균)
        overall_score = (sentiment_ratios['Positive'] + sum(category_scores.values()) / len(categories)) / 2

        # 결과 저장
        store_summaries.append({
            'Store Name': store_name,
            'Summary': refined_summary,
            'Negative Ratio': sentiment_ratios['Negative'],
            'Neutral Ratio': sentiment_ratios['Neutral'],
            'Positive Ratio': sentiment_ratios['Positive'],
            'Taste Score': category_scores.get('맛', 'N/A'),
            'Price Score': category_scores.get('가격', 'N/A'),
            'Service Score': category_scores.get('서비스', 'N/A'),
            'Atmosphere Score': category_scores.get('분위기', 'N/A'),
            'Recommendation Score': category_scores.get('추천', 'N/A'),
            'Non-recommendation Score': category_scores.get('비추천', 'N/A'),
            'Overall Score': overall_score,
            'Top Menus': ', '.join([menu for menu, count in top_menus])  # 대표 메뉴 추가
        })

    except Exception as e:
        print(f"Error processing store {store_name}: {e}")

# 현재 날짜와 시간을 가져와서 파일 이름에 포함
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = f'{current_time}_리뷰요약및점수.csv'

# 결과를 CSV 파일로 저장
try:
    final_results = pd.DataFrame(store_summaries)
    final_results.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}")
except Exception as e:
    print(f"Error in saving results to CSV: {e}")


Processing Stores:   0%|          | 0/737 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores:   0%|          | 2/737 [00:01<06:49,  1.79it/s]

Error processing store (주)플레이버타운: Input length of input_ids is 218, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores:   1%|          | 4/737 [00:02<06:16,  1.95it/s]

Error processing store 72420 미아사거리점: Input length of input_ids is 184, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores:   1%|          | 5/737 [00:03<08:01,  1.52it/s]

...

Error processing store 훅트포케 목동점: Input length of input_ids is 206, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores:  99%|█████████▉| 730/737 [08:09<00:07,  1.09s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores:  99%|█████████▉| 731/737 [08:10<00:06,  1.05s/it]

Error processing store 훅트포케(양재점): Input length of input_ids is 212, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores: 100%|██████████| 737/737 [08:11<00:00,  1.50it/s]

Error processing store 힛힛: Input length of input_ids is 196, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
Results saved to 20240902_183925_리뷰요약및점수.csv


In [12]:
# 가게별 리뷰 종합 요약 및 점수 계산

# 그룹화된 데이터를 가게별로 가져오기
store_grouped = df.groupby('store_name')['content']

store_summaries = []

# 순차적으로 가게별 리뷰를 요약 생성 및 점수 계산
for store_name, reviews in tqdm(store_grouped, desc="Processing Stores"):
    try:
        reviews = reviews.tolist()  # 그룹화된 데이터는 시리즈로 나오므로 리스트로 변환
        num_reviews = len(reviews)

        # 리뷰 전처리
        preprocessed_reviews = [preprocess_text(review) for review in reviews]

        # 리뷰 요약
        summary = summarize_reviews(preprocessed_reviews)
        
        # 요약된 문장을 자연스럽게 다듬음
        refined_summary = refine_summary(summary)

        # 리뷰가 50개 미만인 경우 부족한 데이터에 대한 안내 추가
        if num_reviews < 50:
            refined_summary += " (리뷰가 50개 미만이어서 데이터가 부족할 수 있습니다.)"
        
        # 카테고리별 리뷰 요약 생성
        category_summaries = {}
        for category in ['맛', '가격', '서비스', '분위기']:
            category_reviews = [review for review in preprocessed_reviews if category in review]
            if category_reviews:
                category_summary = summarize_reviews(category_reviews)
                refined_category_summary = refine_summary(category_summary)
                category_summaries[category] = refined_category_summary
            else:
                category_summaries[category] = "관련 리뷰가 충분하지 않습니다."

        # 감성 비율 계산
        sentiments = [sentiment_results[i] for i, r in enumerate(preprocessed_data) if r in reviews]
        sentiment_counts = Counter(sentiments)
        total_reviews = len(sentiments)
        sentiment_ratios = {}

        # 'Negative', 'Neutral', 'Positive' 레이블에 해당하는 인덱스 값을 설정합니다.
        labels = ['Negative', 'Neutral', 'Positive']

        # 각 레이블에 대해 루프를 돌며 감정 비율을 계산합니다.
        for i, label in enumerate(labels):
            # 특정 감정의 리뷰 수를 가져옵니다. 없다면 0을 반환합니다.
            count = sentiment_counts.get(i, 0)
            
            # 전체 리뷰 수에 대한 해당 감정의 비율을 계산합니다.
            ratio = count / total_reviews if total_reviews > 0 else 0
            
            # 계산된 비율을 해당 레이블에 맞춰 딕셔너리에 추가합니다.
            sentiment_ratios[label] = ratio

        # 카테고리별 점수 계산
        category_scores = {}
        for category in categories:
            category_count = sum(category_sentiments[category].values())
            category_scores[category] = category_sentiments[category][2] / category_count if category_count > 0 else 0  # 긍정 점수

        # 대표 메뉴 추출
        top_menus = extract_top_menus(preprocessed_reviews)

        # 최종 점수 계산 (예: 감성 비율과 카테고리 점수의 평균)
        overall_score = (sentiment_ratios['Positive'] + sum(category_scores.values()) / len(categories)) / 2

        # 결과 저장
        store_summaries.append({
            'Store Name': store_name,
            'Summary': refined_summary,
            'Negative Ratio': sentiment_ratios['Negative'],
            'Neutral Ratio': sentiment_ratios['Neutral'],
            'Positive Ratio': sentiment_ratios['Positive'],
            'Taste Score': category_scores.get('맛', 'N/A'),
            'Price Score': category_scores.get('가격', 'N/A'),
            'Service Score': category_scores.get('서비스', 'N/A'),
            'Atmosphere Score': category_scores.get('분위기', 'N/A'),
            'Recommendation Score': category_scores.get('추천', 'N/A'),
            'Non-recommendation Score': category_scores.get('비추천', 'N/A'),
            'Overall Score': overall_score,
            'Top Menus': ', '.join([menu for menu, count in top_menus]) if top_menus else 'N/A',
            'Taste Summary': category_summaries['맛'],
            'Price Summary': category_summaries['가격'],
            'Service Summary': category_summaries['서비스'],
            'Atmosphere Summary': category_summaries['분위기']
        })

    except Exception as e:
        print(f"Error processing store {store_name}: {e}")

# 현재 날짜와 시간을 가져와서 파일 이름에 포함
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = f'{current_time}_리뷰요약및점수.csv'

# 결과를 CSV 파일로 저장
try:
    final_results = pd.DataFrame(store_summaries)
    final_results.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}")
except Exception as e:
    print(f"Error in saving results to CSV: {e}")


Processing Stores:   0%|          | 0/737 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores:   0%|          | 1/737 [00:01<14:12,  1.16s/it]

Error processing store (주)애브릿지: Input length of input_ids is 184, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores:   0%|          | 2/737 [00:02<12:58,  1.06s/it]

Error processing store (주)플레이버타운: Input length of input_ids is 218, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores:   0%|          | 3/737 [00:03<12:47,  1.05s/it]

...

Error processing store 흑석호치민: Input length of input_ids is 216, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores: 100%|█████████▉| 736/737 [11:48<00:00,  1.03it/s]

Error processing store 히타나: Input length of input_ids is 238, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing Stores: 100%|██████████| 737/737 [11:49<00:00,  1.04it/s]

Error processing store 힛힛: Input length of input_ids is 196, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
Results saved to 20240902_185727_리뷰요약및점수.csv
